In [8]:
%load_ext autoreload
%autoreload 2

from llm_experiments.slack_tools import SlackTools
from llm_experiments.llm import create_model
from langchain_core.tools import tool

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
slack_tools = SlackTools()
slack_tools.post_message(channel="test", text="hello")

message sent to test: hello


(True, 'C08LKPFTGMV', '1744037104.731049')

In [6]:
@tool
def post(channel: str, text: str):
    """Post a message to a Slack channel."""
    slack_tools.post_message(channel=channel, text=text)

In [10]:
model = create_model().bind_tools([post])
res = model.invoke("post hello on test channel")
res.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  post (call_jqlNHhcp1v8SjMgbILKYNBWa)
 Call ID: call_jqlNHhcp1v8SjMgbILKYNBWa
  Args:
    channel: test
    text: hello


In [ ]:
res.tool_calls[-1]["args"]
post.invoke(res.tool_calls[-1])

TypeError: BaseTool.invoke() missing 1 required positional argument: 'input'